# Visualize mask

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import glob

In [10]:
def carla_mask_to_CityScapes(rgb_img : np.array):
    city_scape_mask = np.empty_like(rgb_img)
    classes = {
        0: (0, 0, 0),         # Unlabeled
        # 1: (70, 70, 70),      # Buildings
        1: (128, 64, 128),
        2: (100, 40, 40),     # Fences
        3: (55, 90, 80),      # Other
        4: (220, 20, 60),     # Pedestrians
        5: (153, 153, 153),   # Pole
        6: (157, 234, 50),    # RoadLine
        7: (128, 64, 128),    # Road
        8: (244, 35, 232),    # Sidewalk
        9: (107, 142, 35),    # Vegetation
        10: (0, 0, 142),      # Vehicles
        11: (102, 102, 156),  # Wall
        12: (220, 220, 0),    # TrafficSigns
        13: (70, 130, 180),   # Sky
        14: (81, 0, 81),      # Ground
        15: (150, 100, 100),  # Bridge
        16: (230, 150, 140),  # RailTrack
        17: (180, 165, 180),  # GuardRail
        18: (250, 170, 30),   # TrafficLight
        19: (110, 190, 160),  # Static
        20: (170, 120, 50),   # Dynamic
        21: (45, 60, 150),    # Water
        22: (145, 170, 100)   # Terrain
    }
    for key, value in classes.items():
        city_scape_mask[rgb_img[:,:,0] == key] = value
    return city_scape_mask

# Fix file missing

In [30]:
file_list = glob.glob("_out/rgb/*")
mask_list = glob.glob("_out/segmentation/*")

In [37]:
deleted = 0
for f in file_list:
    full_path = f"_out/segmentation/{os.path.basename(f)}"
    if not os.path.isfile(full_path):
        os.remove(f)
        deleted += 1

print(deleted)

16


In [38]:
deleted = 0
for f in mask_list:
    full_path = f"_out/rgb/{os.path.basename(f)}"
    if not os.path.isfile(full_path):
        os.remove(f)
        deleted += 1

print(deleted)

43


# Process mask

In [7]:
def process_mask(rgb_mask : np.array):
    mask = rgb_mask[:,:,0].copy()
    # convert RoadLine and Road to 1
    mask[np.logical_or(mask == 6, mask == 7)] = 1
    # else convert to 0
    mask[mask != 1] = 0
    return mask


In [15]:
def keep_biggest_mask(input_mask):
    out = np.zeros_like(input_mask)
    contours, _ = cv2.findContours(input_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_area_max = 0
    max_index = 0
    for i, val in enumerate(contours):
        val = cv2.contourArea(val)
        if val > contour_area_max:
            contour_area_max = val
            max_index = i

    return cv2.drawContours(out, contours, max_index, 1, cv2.FILLED)

In [143]:
mask_list = glob.glob("_out/segmentation/*")
for mask_name in mask_list:
    img = cv2.cvtColor(cv2.imread(mask_name), cv2.COLOR_BGR2RGB)
    mask_processed = process_mask(img)
    mask_processed = keep_biggest_mask(mask_processed) 
    # mask_processed = mask_processed * 255
    cv2.imwrite(f"_out/segmentation_processed/{os.path.basename(mask_name)}", mask_processed)
    